# 載入套件

In [1]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
import keras
import numpy as np
from six.moves import range

Using TensorFlow backend.


# 加法器

### Parameters Config 參數設定

設定shell輸出文字的顏色。另外還能設定輸出文字的背景、底線、粗體等(參考:http://inpega.blogspot.com/2015/07/shell.html)

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000             #總樣本數
DIGITS = 3                        #所產生數字的最大位數
REVERSE = False                   #是否將數字從尾到頭反過來寫
MAXLEN = DIGITS + 1 + DIGITS      #兩個數字相加的字串最長長度
chars = '0123456789+ '            #訓練時會出現的所有字串
RNN = layers.LSTM                 #將RNN設定為LSTM
HIDDEN_SIZE = 128                 #隱藏層Neoren數
BATCH_SIZE = 128                  #Batch數
LAYERS = 1                        #層數

#### 整理個別字串成dictionary形式/對字串做encoding/對字串做decoding

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

### Data Generation 資料產生

In [7]:
questions = []       #questions: 兩個數字相加
expected = []        #expected:  兩個數字相加的和
seen = set()         #檢查新產生的數字組合是否出現過
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    # 隨機產生最高三位數的數字(0~999)
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:                        #若新產生的數字組合已經出現過了，則不加入樣本中
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)               #將題目轉成字串形式
    query = q + ' ' * (MAXLEN - len(q))    #補上最後面的空格，使所有題目的字串長度一致
    ans = str(a + b)                       #將答案轉成字串形式
    ans += ' ' * (DIGITS + 1 - len(ans))   #補上最後面的空格，使所有答案的字串長度一致
    if REVERSE:                            #若REVERSE=TRUE，則將兩數相加的字串反過來寫
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
#檢查產生出的資料是否正確合理
print(questions[:5], expected[:5])

['944+3  ', '8+52   ', '516+5  ', '163+8  ', '26+204 '] ['947 ', '60  ', '521 ', '171 ', '230 ']


### Processing 資料前處理

#### 將題目字串及答案字串重新編碼

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


#### 將所有資料分成1/4的訓練資料和3/4的測試資料，在訓練資料中再分成90%的訓練集與10%的驗證集

In [10]:
#隨機打散
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

#把training data的90%作為訓練集，10%作為驗證集
split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

#分別印出訓練集、驗證集及測試集的x和y的大小
print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


#### 查看輸入及輸出資料格式

In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False  True False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False  True False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False  True False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False  True False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False False False False False False Fal

### Build Model 模型建立

In [12]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

############################################

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


### Training 模型訓練

In [13]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    #隨機取10筆樣本印出來，看有沒有回答正確
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 348us/step - loss: 2.0178 - acc: 0.2892 - val_loss: 1.8629 - val_acc: 0.3289
Q 780+57  T 837  ☒ 110 
Q 790+0   T 790  ☒ 100 
Q 810+56  T 866  ☒ 120 
Q 21+81   T 102  ☒ 13  
Q 425+77  T 502  ☒ 100 
Q 909+36  T 945  ☒ 110 
Q 42+389  T 431  ☒ 120 
Q 575+693 T 1268 ☒ 1101
Q 223+93  T 316  ☒ 120 
Q 318+708 T 1026 ☒ 110 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 7s 371us/step - loss: 1.8465 - acc: 0.3312 - val_loss: 1.8527 - val_acc: 0.3280
Q 61+8    T 69   ☒ 34  
Q 88+47   T 135  ☒ 107 
Q 712+420 T 1132 ☒ 104 
Q 414+312 T 726  ☒ 124 
Q 575+693 T 1268 ☒ 1117
Q 438+402 T 840  ☒ 111 
Q 20+225  T 245  ☒ 154 
Q 316+27  T 343  ☒ 154 
Q 532+725 T 1257 ☒ 111 
Q 75+155  T 230  ☒ 107 

-------------------

18000/18000 [==============================] - 7s 389us/step - loss: 1.1968 - acc: 0.5537 - val_loss: 1.1992 - val_acc: 0.5485
Q 34+912  T 946  ☒ 943 
Q 649+156 T 805  ☒ 711 
Q 475+950 T 1425 ☒ 1436
Q 190+62  T 252  ☒ 241 
Q 199+165 T 364  ☒ 216 
Q 23+441  T 464  ☒ 465 
Q 479+1   T 480  ☒ 560 
Q 98+333  T 431  ☑ 431 
Q 94+996  T 1090 ☒ 1055
Q 50+373  T 423  ☒ 416 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 7s 412us/step - loss: 1.1497 - acc: 0.5720 - val_loss: 1.1552 - val_acc: 0.5681
Q 35+660  T 695  ☒ 699 
Q 176+32  T 208  ☒ 291 
Q 296+24  T 320  ☒ 322 
Q 1+946   T 947  ☑ 947 
Q 311+157 T 468  ☒ 411 
Q 177+307 T 484  ☒ 475 
Q 88+121  T 209  ☒ 290 
Q 7+243   T 250  ☒ 232 
Q 675+57  T 732  ☑ 732 
Q 787+780 T 1567 ☒ 1545

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 11s 634us/step - loss: 0.4798 - acc: 0.8446 - val_loss: 0.5283 - val_acc: 0.8126
Q 75+292  T 367  ☑ 367 
Q 19+511  T 530  ☒ 518 
Q 67+793  T 860  ☑ 860 
Q 950+939 T 1889 ☒ 1890
Q 5+405   T 410  ☒ 411 
Q 130+21  T 151  ☒ 222 
Q 923+34  T 957  ☒ 967 
Q 583+11  T 594  ☑ 594 
Q 50+218  T 268  ☑ 268 
Q 953+936 T 1889 ☒ 1899

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 9s 499us/step - loss: 0.4410 - acc: 0.8595 - val_loss: 0.4939 - val_acc: 0.8191
Q 642+872 T 1514 ☒ 1513
Q 223+880 T 1103 ☑ 1103
Q 349+10  T 359  ☑ 359 
Q 68+447  T 515  ☑ 515 
Q 69+348  T 417  ☒ 416 
Q 551+3   T 554  ☑ 554 
Q 51+488  T 539  ☑ 539 
Q 898+509 T 1407 ☒ 1417
Q 25+84   T 109  ☑ 109 
Q 99+817  T 916  ☒ 917 

--------------------------------------------------
Iteration 30
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [====

18000/18000 [==============================] - 5s 273us/step - loss: 0.1638 - acc: 0.9653 - val_loss: 0.2190 - val_acc: 0.9270
Q 833+847 T 1680 ☒ 1670
Q 90+851  T 941  ☑ 941 
Q 51+198  T 249  ☑ 249 
Q 621+149 T 770  ☒ 760 
Q 435+193 T 628  ☒ 638 
Q 223+880 T 1103 ☑ 1103
Q 523+704 T 1227 ☑ 1227
Q 2+814   T 816  ☑ 816 
Q 80+881  T 961  ☑ 961 
Q 693+39  T 732  ☑ 732 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 213us/step - loss: 0.1662 - acc: 0.9591 - val_loss: 0.2145 - val_acc: 0.9301
Q 246+80  T 326  ☑ 326 
Q 989+2   T 991  ☑ 991 
Q 948+176 T 1124 ☑ 1124
Q 208+50  T 258  ☑ 258 
Q 771+55  T 826  ☑ 826 
Q 360+887 T 1247 ☑ 1247
Q 504+758 T 1262 ☑ 1262
Q 13+424  T 437  ☒ 447 
Q 46+3    T 49   ☒ 59  
Q 672+34  T 706  ☑ 706 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 9s 512us/step - loss: 0.0889 - acc: 0.9793 - val_loss: 0.2022 - val_acc: 0.9281
Q 118+274 T 392  ☑ 392 
Q 224+29  T 253  ☑ 253 
Q 27+328  T 355  ☑ 355 
Q 426+895 T 1321 ☑ 1321
Q 268+444 T 712  ☑ 712 
Q 687+38  T 725  ☑ 725 
Q 611+12  T 623  ☑ 623 
Q 21+515  T 536  ☑ 536 
Q 787+212 T 999  ☒ 909 
Q 719+516 T 1235 ☑ 1235

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 462us/step - loss: 0.1037 - acc: 0.9712 - val_loss: 0.1419 - val_acc: 0.9509
Q 269+54  T 323  ☑ 323 
Q 67+671  T 738  ☒ 739 
Q 33+59   T 92   ☒ 90  
Q 494+483 T 977  ☒ 967 
Q 387+428 T 815  ☑ 815 
Q 294+362 T 656  ☑ 656 
Q 146+107 T 253  ☑ 253 
Q 447+733 T 1180 ☒ 1170
Q 472+910 T 1382 ☑ 1382
Q 984+973 T 1957 ☑ 1957

--------------------------------------------------
Iteration 58
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 7s 403us/step - loss: 0.0674 - acc: 0.9819 - val_loss: 0.1027 - val_acc: 0.9634
Q 45+319  T 364  ☑ 364 
Q 48+257  T 305  ☑ 305 
Q 794+635 T 1429 ☑ 1429
Q 820+54  T 874  ☑ 874 
Q 87+808  T 895  ☑ 895 
Q 712+420 T 1132 ☑ 1132
Q 11+840  T 851  ☒ 841 
Q 6+931   T 937  ☑ 937 
Q 656+158 T 814  ☑ 814 
Q 122+869 T 991  ☑ 991 

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 348us/step - loss: 0.0273 - acc: 0.9977 - val_loss: 0.0755 - val_acc: 0.9750
Q 84+590  T 674  ☑ 674 
Q 403+759 T 1162 ☑ 1162
Q 389+264 T 653  ☑ 653 
Q 816+320 T 1136 ☑ 1136
Q 166+41  T 207  ☑ 207 
Q 298+796 T 1094 ☒ 1084
Q 288+64  T 352  ☑ 352 
Q 634+44  T 678  ☑ 678 
Q 26+23   T 49   ☒ 59  
Q 388+45  T 433  ☑ 433 

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 4s 246us/step - loss: 0.0119 - acc: 0.9998 - val_loss: 0.0649 - val_acc: 0.9797
Q 40+19   T 59   ☑ 59  
Q 34+370  T 404  ☑ 404 
Q 14+833  T 847  ☑ 847 
Q 435+8   T 443  ☑ 443 
Q 373+78  T 451  ☑ 451 
Q 700+4   T 704  ☑ 704 
Q 574+804 T 1378 ☑ 1378
Q 17+11   T 28   ☒ 37  
Q 548+51  T 599  ☑ 599 
Q 52+904  T 956  ☑ 956 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 351us/step - loss: 0.0109 - acc: 0.9998 - val_loss: 0.0639 - val_acc: 0.9786
Q 723+22  T 745  ☑ 745 
Q 788+40  T 828  ☑ 828 
Q 13+654  T 667  ☑ 667 
Q 2+956   T 958  ☑ 958 
Q 277+3   T 280  ☑ 280 
Q 34+942  T 976  ☑ 976 
Q 389+53  T 442  ☑ 442 
Q 272+29  T 301  ☑ 301 
Q 478+595 T 1073 ☑ 1073
Q 807+137 T 944  ☒ 934 

--------------------------------------------------
Iteration 86
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 337us/step - loss: 0.0066 - acc: 0.9999 - val_loss: 0.0546 - val_acc: 0.9824
Q 690+60  T 750  ☑ 750 
Q 19+137  T 156  ☑ 156 
Q 39+835  T 874  ☑ 874 
Q 734+736 T 1470 ☒ 1460
Q 938+32  T 970  ☒ 960 
Q 84+360  T 444  ☑ 444 
Q 156+407 T 563  ☑ 563 
Q 475+950 T 1425 ☑ 1425
Q 664+13  T 677  ☑ 677 
Q 317+22  T 339  ☑ 339 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 442us/step - loss: 0.0062 - acc: 0.9999 - val_loss: 0.0565 - val_acc: 0.9826
Q 54+748  T 802  ☑ 802 
Q 91+908  T 999  ☑ 999 
Q 952+35  T 987  ☑ 987 
Q 416+641 T 1057 ☑ 1057
Q 321+2   T 323  ☑ 323 
Q 971+755 T 1726 ☑ 1726
Q 831+30  T 861  ☑ 861 
Q 438+402 T 840  ☑ 840 
Q 658+738 T 1396 ☑ 1396
Q 124+36  T 160  ☑ 160 


### Testing 模型測試

In [14]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
right = 0
predictions = model.predict_classes(test_x, verbose = 1) 
for i in range(60000):
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(predictions[i], calc_argmax=False)
    if correct == guess:
        right = right + 1

acc = right/60000
print("Accuracy : ", acc)
#####################################################

MSG : Prediction
60000/60000 [==============================] - 12s 203us/step
Accuracy :  0.9351833333333334


Validation Accuracy為0.9826，Test Accuracy為0.9352，可以發現模型配得還不錯。

***

# 減法器

### Parameters Config

In [15]:
chars = '0123456789- '
ctable = CharacterTable(chars)
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

### Data Generation

#### 產生資料方式跟加法器相同
#### 唯一差別是:由於規定a必須大於等於b，使相減後的差大於等於0，故將a小於b的組合作a,b互換的動作

In [16]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    #若a小於b，則令a、b互換
    if a < b :
        a = a + b
        b = a - b
        a = a - b
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total substraction questions:', len(questions))

Generating data...
Total substraction questions: 80000


In [17]:
print(questions[:5], expected[:5])

['9-5    ', '107-14 ', '272-19 ', '8-0    ', '293-1  '] ['4   ', '93  ', '253 ', '8   ', '292 ']


### Processing

In [18]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [19]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [20]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False  True False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False False False False False False False False  True False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False  True
   False]
  [False False False False False  True False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False False False False  True Fal

### Build Model

In [21]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

############################################

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


### Training

模型在減法器的效果沒有很好，故將迭代次數調高至150

In [22]:
for iteration in range(150):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 463us/step - loss: 1.8957 - acc: 0.3212 - val_loss: 1.7348 - val_acc: 0.3546
Q 899-301 T 598  ☒ 13  
Q 290-48  T 242  ☒ 13  
Q 958-391 T 567  ☒ 13  
Q 536-55  T 481  ☒ 13  
Q 579-4   T 575  ☒ 13  
Q 374-42  T 332  ☒ 13  
Q 611-568 T 43   ☒ 13  
Q 175-45  T 130  ☒ 13  
Q 303-4   T 299  ☒ 13  
Q 543-72  T 471  ☒ 13  

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 302us/step - loss: 1.7196 - acc: 0.3549 - val_loss: 1.7137 - val_acc: 0.3584
Q 844-202 T 642  ☒ 131 
Q 596-45  T 551  ☒ 130 
Q 674-477 T 197  ☒ 131 
Q 377-180 T 197  ☒ 131 
Q 944-27  T 917  ☒ 131 
Q 890-720 T 170  ☒ 160 
Q 734-508 T 226  ☒ 331 
Q 732-2   T 730  ☒ 131 
Q 159-53  T 106  ☒ 131 
Q 994-547 T 447  ☒ 160 

-------------------

18000/18000 [==============================] - 10s 535us/step - loss: 1.0989 - acc: 0.6056 - val_loss: 1.1067 - val_acc: 0.5881
Q 430-51  T 379  ☒ 375 
Q 897-858 T 39   ☒ 10  
Q 795-22  T 773  ☒ 772 
Q 166-58  T 108  ☒ 110 
Q 141-108 T 33   ☒ 13  
Q 659-38  T 621  ☒ 625 
Q 177-13  T 164  ☒ 155 
Q 993-25  T 968  ☒ 953 
Q 630-73  T 557  ☒ 555 
Q 521-18  T 503  ☒ 505 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 422us/step - loss: 1.0672 - acc: 0.6149 - val_loss: 1.0660 - val_acc: 0.6093
Q 300-71  T 229  ☒ 246 
Q 546-519 T 27   ☒ 1   
Q 632-73  T 559  ☒ 567 
Q 937-412 T 525  ☒ 583 
Q 439-80  T 359  ☒ 355 
Q 815-5   T 810  ☒ 813 
Q 944-39  T 905  ☒ 906 
Q 194-20  T 174  ☒ 171 
Q 591-80  T 511  ☑ 511 
Q 219-58  T 161  ☒ 164 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [====

18000/18000 [==============================] - 6s 323us/step - loss: 0.7640 - acc: 0.7236 - val_loss: 0.7859 - val_acc: 0.7102
Q 671-603 T 68   ☒ 77  
Q 249-43  T 206  ☒ 202 
Q 515-43  T 472  ☒ 471 
Q 323-27  T 296  ☑ 296 
Q 614-4   T 610  ☒ 618 
Q 616-52  T 564  ☑ 564 
Q 153-70  T 83   ☒ 73  
Q 670-440 T 230  ☒ 237 
Q 777-713 T 64   ☒ 15  
Q 297-231 T 66   ☒ 15  

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 7s 364us/step - loss: 0.7412 - acc: 0.7292 - val_loss: 0.7634 - val_acc: 0.7127
Q 655-64  T 591  ☒ 590 
Q 308-9   T 299  ☒ 290 
Q 947-263 T 684  ☒ 781 
Q 583-81  T 502  ☒ 500 
Q 784-522 T 262  ☒ 341 
Q 942-310 T 632  ☒ 630 
Q 683-268 T 415  ☒ 406 
Q 754-38  T 716  ☑ 716 
Q 675-91  T 584  ☒ 587 
Q 655-64  T 591  ☒ 590 

--------------------------------------------------
Iteration 30
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 316us/step - loss: 0.3593 - acc: 0.8877 - val_loss: 0.3952 - val_acc: 0.8640
Q 980-2   T 978  ☑ 978 
Q 290-175 T 115  ☒ 104 
Q 510-33  T 477  ☑ 477 
Q 223-71  T 152  ☒ 143 
Q 208-80  T 128  ☑ 128 
Q 870-46  T 824  ☑ 824 
Q 822-784 T 38   ☒ 49  
Q 924-64  T 860  ☑ 860 
Q 509-51  T 458  ☑ 458 
Q 389-41  T 348  ☑ 348 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 323us/step - loss: 0.3375 - acc: 0.8984 - val_loss: 0.3773 - val_acc: 0.8715
Q 187-27  T 160  ☑ 160 
Q 222-0   T 222  ☑ 222 
Q 836-48  T 788  ☑ 788 
Q 344-94  T 250  ☑ 250 
Q 733-697 T 36   ☒ 55  
Q 719-62  T 657  ☑ 657 
Q 326-62  T 264  ☑ 264 
Q 329-35  T 294  ☑ 294 
Q 368-38  T 330  ☒ 331 
Q 862-16  T 846  ☑ 846 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 9s 496us/step - loss: 0.1586 - acc: 0.9593 - val_loss: 0.1917 - val_acc: 0.9376
Q 795-282 T 513  ☑ 513 
Q 689-82  T 607  ☑ 607 
Q 202-7   T 195  ☑ 195 
Q 680-78  T 602  ☑ 602 
Q 709-6   T 703  ☑ 703 
Q 306-55  T 251  ☑ 251 
Q 675-25  T 650  ☑ 650 
Q 26-8    T 18   ☑ 18  
Q 175-45  T 130  ☑ 130 
Q 883-9   T 874  ☑ 874 

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 7s 410us/step - loss: 0.1467 - acc: 0.9642 - val_loss: 0.2029 - val_acc: 0.9326
Q 628-368 T 260  ☒ 250 
Q 675-520 T 155  ☑ 155 
Q 206-38  T 168  ☑ 168 
Q 189-7   T 182  ☑ 182 
Q 576-95  T 481  ☑ 481 
Q 414-16  T 398  ☑ 398 
Q 647-83  T 564  ☑ 564 
Q 91-59   T 32   ☑ 32  
Q 790-70  T 720  ☑ 720 
Q 380-203 T 177  ☑ 177 

--------------------------------------------------
Iteration 58
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

...

18000/18000 [==============================] - 6s 327us/step - loss: 0.0734 - acc: 0.9796 - val_loss: 0.1006 - val_acc: 0.9646
Q 656-56  T 600  ☑ 600 
Q 300-71  T 229  ☑ 229 
Q 935-36  T 899  ☒ 999 
Q 636-353 T 283  ☑ 283 
Q 280-63  T 217  ☑ 217 
Q 481-75  T 406  ☑ 406 
Q 561-430 T 131  ☑ 131 
Q 33-1    T 32   ☒ 31  
Q 448-392 T 56   ☒ 66  
Q 964-16  T 948  ☑ 948 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 469us/step - loss: 0.0443 - acc: 0.9916 - val_loss: 0.0880 - val_acc: 0.9704
Q 836-97  T 739  ☑ 739 
Q 946-622 T 324  ☑ 324 
Q 891-72  T 819  ☑ 819 
Q 833-7   T 826  ☑ 826 
Q 556-62  T 494  ☑ 494 
Q 91-47   T 44   ☑ 44  
Q 698-83  T 615  ☑ 615 
Q 956-705 T 251  ☑ 251 
Q 299-59  T 240  ☑ 240 
Q 237-18  T 219  ☑ 219 

--------------------------------------------------
Iteration 86
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 188us/step - loss: 0.0291 - acc: 0.9951 - val_loss: 0.0728 - val_acc: 0.9758
Q 991-25  T 966  ☑ 966 
Q 940-20  T 920  ☑ 920 
Q 226-72  T 154  ☑ 154 
Q 403-155 T 248  ☑ 248 
Q 600-191 T 409  ☑ 409 
Q 514-72  T 442  ☑ 442 
Q 763-179 T 584  ☑ 584 
Q 251-54  T 197  ☑ 197 
Q 867-0   T 867  ☑ 867 
Q 161-15  T 146  ☑ 146 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 214us/step - loss: 0.0264 - acc: 0.9960 - val_loss: 0.0728 - val_acc: 0.9766
Q 86-36   T 50   ☑ 50  
Q 945-26  T 919  ☑ 919 
Q 619-14  T 605  ☑ 605 
Q 334-55  T 279  ☑ 279 
Q 921-833 T 88   ☑ 88  
Q 499-55  T 444  ☑ 444 
Q 196-21  T 175  ☑ 175 
Q 634-47  T 587  ☑ 587 
Q 952-72  T 880  ☑ 880 
Q 546-75  T 471  ☑ 471 

--------------------------------------------------
Iteration 100
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [====

18000/18000 [==============================] - 7s 381us/step - loss: 0.0328 - acc: 0.9926 - val_loss: 0.0726 - val_acc: 0.9761
Q 733-647 T 86   ☑ 86  
Q 692-63  T 629  ☑ 629 
Q 939-145 T 794  ☑ 794 
Q 753-430 T 323  ☑ 323 
Q 561-220 T 341  ☑ 341 
Q 812-213 T 599  ☒ 609 
Q 953-32  T 921  ☑ 921 
Q 962-17  T 945  ☑ 945 
Q 876-785 T 91   ☑ 91  
Q 558-345 T 213  ☑ 213 

--------------------------------------------------
Iteration 113
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 347us/step - loss: 0.0192 - acc: 0.9975 - val_loss: 0.0703 - val_acc: 0.9766
Q 375-0   T 375  ☑ 375 
Q 145-81  T 64   ☑ 64  
Q 574-295 T 279  ☑ 279 
Q 896-45  T 851  ☑ 851 
Q 42-23   T 19   ☑ 19  
Q 752-21  T 731  ☑ 731 
Q 88-46   T 42   ☑ 42  
Q 941-90  T 851  ☑ 851 
Q 867-576 T 291  ☑ 291 
Q 914-24  T 890  ☒ 880 

--------------------------------------------------
Iteration 114
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [===

18000/18000 [==============================] - 6s 321us/step - loss: 0.0143 - acc: 0.9985 - val_loss: 0.0738 - val_acc: 0.9746
Q 391-192 T 199  ☒ 198 
Q 790-70  T 720  ☑ 720 
Q 364-90  T 274  ☑ 274 
Q 600-191 T 409  ☑ 409 
Q 977-4   T 973  ☑ 973 
Q 967-334 T 633  ☑ 633 
Q 719-671 T 48   ☒ 58  
Q 987-31  T 956  ☑ 956 
Q 972-52  T 920  ☑ 920 
Q 892-811 T 81   ☒ 71  

--------------------------------------------------
Iteration 127
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 277us/step - loss: 0.0424 - acc: 0.9893 - val_loss: 0.5277 - val_acc: 0.8730
Q 633-355 T 278  ☑ 278 
Q 116-3   T 113  ☑ 113 
Q 532-85  T 447  ☑ 447 
Q 698-83  T 615  ☒ 605 
Q 725-102 T 623  ☑ 623 
Q 736-733 T 3    ☑ 3   
Q 747-33  T 714  ☑ 714 
Q 213-69  T 144  ☑ 144 
Q 917-807 T 110  ☒ 11  
Q 453-77  T 376  ☒ 366 

--------------------------------------------------
Iteration 128
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [===

18000/18000 [==============================] - 6s 352us/step - loss: 0.0459 - acc: 0.9879 - val_loss: 0.3363 - val_acc: 0.9023
Q 664-27  T 637  ☑ 637 
Q 468-16  T 452  ☒ 442 
Q 902-137 T 765  ☒ 775 
Q 326-52  T 274  ☑ 274 
Q 170-6   T 164  ☒ 165 
Q 848-733 T 115  ☑ 115 
Q 205-44  T 161  ☑ 161 
Q 609-0   T 609  ☑ 609 
Q 979-90  T 889  ☒ 899 
Q 867-589 T 278  ☑ 278 

--------------------------------------------------
Iteration 141
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 464us/step - loss: 0.1000 - acc: 0.9679 - val_loss: 0.0972 - val_acc: 0.9667
Q 570-16  T 554  ☑ 554 
Q 62-62   T 0    ☑ 0   
Q 656-65  T 591  ☑ 591 
Q 879-254 T 625  ☑ 625 
Q 611-128 T 483  ☒ 493 
Q 94-81   T 13   ☑ 13  
Q 723-93  T 630  ☑ 630 
Q 63-20   T 43   ☑ 43  
Q 753-607 T 146  ☑ 146 
Q 414-53  T 361  ☑ 361 

--------------------------------------------------
Iteration 142
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [===

### Testing

In [23]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
right = 0
predictions = model.predict_classes(test_x, verbose = 1) 
for i in range(60000):
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(predictions[i], calc_argmax=False)
    if correct == guess:
        right = right + 1

acc = right/60000
print("Accuracy : ", acc)
#####################################################

MSG : Prediction
60000/60000 [==============================] - 10s 167us/step
Accuracy :  0.9229666666666667


|TRAINING_SIZE|Iterarions|Validation Accuracy|Test Accuracy|
|---|---|---|---|
|80000|100|0.9682|0.8855|
|80000|150|0.9799|0.9230|

由於迭代次數 = 100的模型準確率沒有很高，所以將迭代次數調高至150，準確率也因此提高。

***

# 加法+減法

### Parameters Config

將原本的80000筆樣本分成40000筆加法和40000筆減法去建立模型與訓練會得到蠻低的準確率(約0.58)，故將樣本數增加至160000筆(80000筆加法和80000筆減法)

In [24]:
TRAINING_SIZE = 160000
chars = '0123456789+- '
ctable = CharacterTable(chars)
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

### Data Generation

In [25]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < (TRAINING_SIZE/2):
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)

while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    #若a小於b，則令a、b互換
    if a < b :
        a = a + b
        b = a - b
        a = a - b
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)

print('Total addition&substraction questions:', len(questions))

Generating data...
Total addition&substraction questions: 160000


### Processing

In [26]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [27]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = np.concatenate([x[0:20000], x[80000:100000]])
train_y = np.concatenate([y[0:20000], y[80000:100000]])
test_x = np.concatenate([x[20000:80000], x[100000:160000]])
test_y = np.concatenate([y[20000:80000], y[100000:160000]])

#split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = np.concatenate([train_x[0:18000], train_x[20000:38000]]), np.concatenate([train_x[18000:20000], train_x[38000:40000]])
(y_train, y_val) = np.concatenate([train_y[0:18000], train_y[20000:38000]]), np.concatenate([train_y[18000:20000], train_y[38000:40000]])

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(36000, 7, 13)
(36000, 4, 13)
Validation Data:
(4000, 7, 13)
(4000, 4, 13)
Testing Data:
(120000, 7, 13)
(120000, 4, 13)


In [28]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False False False
   False  True]
  [False False False  True False False False False False False False
   False False]
  [False False False False False False False False  True False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False False  True False
   False False]
  [False False False False False False False  True False False False
   False False]
  [False False False False False False False False False  True False
   False False]]

 [[False False False False  True False False False False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [False False False False False False False  True False False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False Fals

### Build Model

In [29]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

############################################

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


### Training

In [30]:
for iteration in range(150):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 13s 360us/step - loss: 1.9015 - acc: 0.3199 - val_loss: 1.7987 - val_acc: 0.3411
Q 485+24  T 509  ☒ 100 
Q 446-264 T 182  ☒ 10  
Q 211-66  T 145  ☒ 10  
Q 334-151 T 183  ☒ 10  
Q 542-64  T 478  ☒ 10  
Q 289+49  T 338  ☒ 101 
Q 892+22  T 914  ☒ 100 
Q 460-85  T 375  ☒ 10  
Q 512+726 T 1238 ☒ 100 
Q 768-754 T 14   ☒ 101 

--------------------------------------------------
Iteration 1
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 12s 334us/step - loss: 1.7725 - acc: 0.3422 - val_loss: 1.7577 - val_acc: 0.3511
Q 495+571 T 1066 ☒ 100 
Q 633+28  T 661  ☒ 405 
Q 651-451 T 200  ☒ 24  
Q 775+23  T 798  ☒ 805 
Q 821-232 T 589  ☒ 24  
Q 809-143 T 666  ☒ 40  
Q 65+145  T 210  ☒ 405 
Q 302+129 T 431  ☒ 104 
Q 804+770 T 1574 ☒ 100 
Q 963-89  T 874  ☒ 485 

-----------------

36000/36000 [==============================] - 17s 463us/step - loss: 1.1333 - acc: 0.5761 - val_loss: 1.1218 - val_acc: 0.5787
Q 895-49  T 846  ☒ 820 
Q 470-11  T 459  ☒ 447 
Q 400+30  T 430  ☒ 452 
Q 586+907 T 1493 ☒ 1465
Q 586+907 T 1493 ☒ 1465
Q 423+44  T 467  ☑ 467 
Q 960-785 T 175  ☒ 179 
Q 102+2   T 104  ☒ 111 
Q 731-451 T 280  ☒ 287 
Q 757-54  T 703  ☒ 710 

--------------------------------------------------
Iteration 15
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 9s 258us/step - loss: 1.0911 - acc: 0.5907 - val_loss: 1.0852 - val_acc: 0.5874
Q 474-298 T 176  ☒ 186 
Q 88+979  T 1067 ☒ 1072
Q 443+658 T 1101 ☒ 1190
Q 455+58  T 513  ☒ 524 
Q 34+431  T 465  ☒ 477 
Q 357+468 T 825  ☒ 811 
Q 749+526 T 1275 ☒ 1279
Q 869-681 T 188  ☒ 199 
Q 79+235  T 314  ☒ 311 
Q 655-499 T 156  ☒ 197 

--------------------------------------------------
Iteration 16
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [====

36000/36000 [==============================] - 10s 277us/step - loss: 0.4810 - acc: 0.8281 - val_loss: 0.4825 - val_acc: 0.8214
Q 849-414 T 435  ☒ 445 
Q 64+382  T 446  ☑ 446 
Q 855-676 T 179  ☒ 188 
Q 710-475 T 235  ☒ 236 
Q 974-696 T 278  ☑ 278 
Q 609+90  T 699  ☒ 690 
Q 851-610 T 241  ☑ 241 
Q 3+195   T 198  ☑ 198 
Q 989-149 T 840  ☒ 839 
Q 688-23  T 665  ☑ 665 

--------------------------------------------------
Iteration 29
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 14s 390us/step - loss: 0.4329 - acc: 0.8498 - val_loss: 0.4454 - val_acc: 0.8352
Q 689+767 T 1456 ☒ 1455
Q 646-147 T 499  ☑ 499 
Q 962-815 T 147  ☑ 147 
Q 937-594 T 343  ☒ 353 
Q 206+379 T 585  ☒ 575 
Q 663-196 T 467  ☑ 467 
Q 31+72   T 103  ☒ 113 
Q 648+101 T 749  ☒ 758 
Q 7+786   T 793  ☒ 783 
Q 916-202 T 714  ☑ 714 

--------------------------------------------------
Iteration 30
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [===

36000/36000 [==============================] - 12s 344us/step - loss: 0.1283 - acc: 0.9697 - val_loss: 0.1783 - val_acc: 0.9420
Q 235+319 T 554  ☑ 554 
Q 927-96  T 831  ☑ 831 
Q 320+9   T 329  ☑ 329 
Q 4+605   T 609  ☑ 609 
Q 99+352  T 451  ☑ 451 
Q 882-56  T 826  ☑ 826 
Q 847-288 T 559  ☑ 559 
Q 159+196 T 355  ☑ 355 
Q 611-133 T 478  ☑ 478 
Q 341-88  T 253  ☑ 253 

--------------------------------------------------
Iteration 43
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 8s 234us/step - loss: 0.1233 - acc: 0.9700 - val_loss: 0.1772 - val_acc: 0.9403
Q 394-27  T 367  ☑ 367 
Q 693+229 T 922  ☒ 912 
Q 773-32  T 741  ☑ 741 
Q 867-803 T 64   ☒ 54  
Q 314-93  T 221  ☑ 221 
Q 24+812  T 836  ☑ 836 
Q 559+44  T 603  ☒ 613 
Q 594-454 T 140  ☑ 140 
Q 66+369  T 435  ☑ 435 
Q 558-250 T 308  ☒ 208 

--------------------------------------------------
Iteration 44
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [====

36000/36000 [==============================] - 13s 363us/step - loss: 0.0490 - acc: 0.9903 - val_loss: 0.1154 - val_acc: 0.9598
Q 208+4   T 212  ☑ 212 
Q 945+28  T 973  ☑ 973 
Q 505+48  T 553  ☑ 553 
Q 956-113 T 843  ☑ 843 
Q 38+917  T 955  ☑ 955 
Q 952+9   T 961  ☑ 961 
Q 895-576 T 319  ☑ 319 
Q 397+119 T 516  ☑ 516 
Q 337+13  T 350  ☒ 340 
Q 77+701  T 778  ☑ 778 

--------------------------------------------------
Iteration 57
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 8s 224us/step - loss: 0.0814 - acc: 0.9759 - val_loss: 0.1013 - val_acc: 0.9653
Q 548+577 T 1125 ☑ 1125
Q 157+664 T 821  ☑ 821 
Q 587-46  T 541  ☑ 541 
Q 102+7   T 109  ☑ 109 
Q 344-92  T 252  ☑ 252 
Q 89+528  T 617  ☑ 617 
Q 3+195   T 198  ☑ 198 
Q 383+635 T 1018 ☑ 1018
Q 58+155  T 213  ☑ 213 
Q 209+47  T 256  ☒ 257 

--------------------------------------------------
Iteration 58
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [====

...

36000/36000 [==============================] - 17s 481us/step - loss: 0.0403 - acc: 0.9885 - val_loss: 0.0666 - val_acc: 0.9781
Q 886+59  T 945  ☑ 945 
Q 244-26  T 218  ☑ 218 
Q 417-69  T 348  ☑ 348 
Q 454-90  T 364  ☑ 364 
Q 823-700 T 123  ☑ 123 
Q 392-38  T 354  ☑ 354 
Q 49+901  T 950  ☑ 950 
Q 54+185  T 239  ☑ 239 
Q 845-14  T 831  ☑ 831 
Q 680+615 T 1295 ☑ 1295

--------------------------------------------------
Iteration 85
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 23s 626us/step - loss: 0.0197 - acc: 0.9965 - val_loss: 0.0646 - val_acc: 0.9785
Q 511-94  T 417  ☑ 417 
Q 513+31  T 544  ☑ 544 
Q 700+444 T 1144 ☑ 1144
Q 2+658   T 660  ☑ 660 
Q 961-98  T 863  ☑ 863 
Q 735-407 T 328  ☑ 328 
Q 288-256 T 32   ☑ 32  
Q 454+492 T 946  ☑ 946 
Q 781+882 T 1663 ☑ 1663
Q 120+7   T 127  ☑ 127 

--------------------------------------------------
Iteration 86
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [===

36000/36000 [==============================] - 9s 251us/step - loss: 0.0453 - acc: 0.9861 - val_loss: 0.0837 - val_acc: 0.9722
Q 586-57  T 529  ☑ 529 
Q 801+255 T 1056 ☒ 1046
Q 802-87  T 715  ☑ 715 
Q 380-52  T 328  ☑ 328 
Q 623-397 T 226  ☑ 226 
Q 794+65  T 859  ☑ 859 
Q 543+128 T 671  ☑ 671 
Q 335-40  T 295  ☑ 295 
Q 545+97  T 642  ☑ 642 
Q 20+530  T 550  ☑ 550 

--------------------------------------------------
Iteration 99
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 10s 268us/step - loss: 0.0445 - acc: 0.9867 - val_loss: 0.0737 - val_acc: 0.9751
Q 297+30  T 327  ☑ 327 
Q 884+702 T 1586 ☑ 1586
Q 883-550 T 333  ☑ 333 
Q 559-30  T 529  ☑ 529 
Q 83+620  T 703  ☑ 703 
Q 945-301 T 644  ☑ 644 
Q 91+464  T 555  ☑ 555 
Q 335-40  T 295  ☒ 395 
Q 906+843 T 1749 ☑ 1749
Q 950-31  T 919  ☑ 919 

--------------------------------------------------
Iteration 100
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [===

36000/36000 [==============================] - 12s 328us/step - loss: 0.0082 - acc: 0.9994 - val_loss: 0.0403 - val_acc: 0.9873
Q 884-450 T 434  ☑ 434 
Q 95+841  T 936  ☑ 936 
Q 183+90  T 273  ☑ 273 
Q 176+77  T 253  ☑ 253 
Q 700+444 T 1144 ☑ 1144
Q 42+583  T 625  ☑ 625 
Q 4+948   T 952  ☑ 952 
Q 45+415  T 460  ☑ 460 
Q 56+608  T 664  ☑ 664 
Q 789+25  T 814  ☑ 814 

--------------------------------------------------
Iteration 113
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 14s 381us/step - loss: 0.0076 - acc: 0.9994 - val_loss: 0.0401 - val_acc: 0.9870
Q 857-180 T 677  ☑ 677 
Q 249-24  T 225  ☑ 225 
Q 906-568 T 338  ☑ 338 
Q 734-370 T 364  ☑ 364 
Q 753-141 T 612  ☑ 612 
Q 861+54  T 915  ☑ 915 
Q 745+347 T 1092 ☑ 1092
Q 680-327 T 353  ☑ 353 
Q 899+15  T 914  ☑ 914 
Q 872-39  T 833  ☑ 833 

--------------------------------------------------
Iteration 114
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [=

36000/36000 [==============================] - 12s 346us/step - loss: 0.0068 - acc: 0.9991 - val_loss: 0.0734 - val_acc: 0.9767
Q 66+61   T 127  ☑ 127 
Q 848-681 T 167  ☑ 167 
Q 611-133 T 478  ☑ 478 
Q 402-94  T 308  ☑ 308 
Q 270+25  T 295  ☑ 295 
Q 713-269 T 444  ☑ 444 
Q 787+892 T 1679 ☑ 1679
Q 98+973  T 1071 ☑ 1071
Q 891-844 T 47   ☑ 47  
Q 988-825 T 163  ☑ 163 

--------------------------------------------------
Iteration 127
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 12s 346us/step - loss: 0.0736 - acc: 0.9770 - val_loss: 0.1350 - val_acc: 0.9567
Q 307-44  T 263  ☑ 263 
Q 24+18   T 42   ☑ 42  
Q 240-42  T 198  ☑ 198 
Q 757+257 T 1014 ☑ 1014
Q 63+956  T 1019 ☒ 1029
Q 816-29  T 787  ☒ 777 
Q 67+371  T 438  ☑ 438 
Q 778-34  T 744  ☑ 744 
Q 966-620 T 346  ☑ 346 
Q 409-325 T 84   ☑ 84  

--------------------------------------------------
Iteration 128
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [=

36000/36000 [==============================] - 17s 477us/step - loss: 0.0039 - acc: 0.9999 - val_loss: 0.0351 - val_acc: 0.9890
Q 650+21  T 671  ☑ 671 
Q 364-97  T 267  ☑ 267 
Q 76+408  T 484  ☑ 484 
Q 590-59  T 531  ☑ 531 
Q 962+416 T 1378 ☑ 1378
Q 633-553 T 80   ☑ 80  
Q 789+25  T 814  ☑ 814 
Q 133+19  T 152  ☑ 152 
Q 587-343 T 244  ☑ 244 
Q 653+58  T 711  ☑ 711 

--------------------------------------------------
Iteration 141
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 20s 546us/step - loss: 0.0039 - acc: 0.9998 - val_loss: 0.0370 - val_acc: 0.9886
Q 781-50  T 731  ☑ 731 
Q 253+20  T 273  ☑ 273 
Q 578+22  T 600  ☑ 600 
Q 72+733  T 805  ☑ 805 
Q 76+658  T 734  ☑ 734 
Q 772+945 T 1717 ☑ 1717
Q 785-95  T 690  ☑ 690 
Q 890-25  T 865  ☑ 865 
Q 523+413 T 936  ☑ 936 
Q 954-293 T 661  ☑ 661 

--------------------------------------------------
Iteration 142
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [=

### Testing

In [31]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
right = 0
predictions = model.predict_classes(test_x, verbose = 1) 
for i in range(60000):
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(predictions[i], calc_argmax=False)
    if correct == guess:
        right = right + 1

acc = right/60000
print("Accuracy : ", acc)
#####################################################

MSG : Prediction
120000/120000 [==============================] - 23s 194us/step
Accuracy :  0.92355


Validation Accuracy為0.9804，Test Accuracy為0.92355，可以看出模型的效果不錯，甚至表現得比只有減法的情形還好。

# 乘法

### 最後來看一下同樣的模型在訓練乘法的資料是不是也會一樣效果不錯吧><

### Parameters Config

In [32]:
TRAINING_SIZE = 80000
chars = '0123456789* '
LAYERS = 2

In [33]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [34]:
ctable = CharacterTable(chars)

In [35]:
ctable.indices_char

{0: ' ',
 1: '*',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

### Data Generation

In [36]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}*{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a * b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)

print('Total multiplication questions:', len(questions))

Generating data...
Total multiplication questions: 80000


In [37]:
print(questions[:5], expected[:5])

['866*43 ', '4*8    ', '4*61   ', '166*956', '17*476 '] ['37238', '32  ', '244 ', '158696', '8092']


### Processing

In [38]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS*2, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS*2)

Vectorization...


In [39]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 6, 12)
Validation Data:
(2000, 7, 12)
(2000, 6, 12)
Testing Data:
(60000, 7, 12)
(60000, 6, 12)


In [40]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False  True False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False  True False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False  True False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False False False  True False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False  Tr

### Build Model

In [41]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS*2))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

############################################

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
lstm_9 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 6, 12)             1548      
Total params: 336,908
Trainable params: 336,908
Non-trainable params: 0
_________________________________________________________________


### Training

In [42]:
for iteration in range(150):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 18s 989us/step - loss: 1.8962 - acc: 0.1357 - val_loss: 1.8133 - val_acc: 0.1572
Q 941*996 T 937236 ☒ 144444
Q 643*207 T 133101 ☒ 144444
Q 410*859 T 352190 ☒ 135500
Q 42*34   T 1428   ☒ 148800
Q 88*432  T 38016  ☒ 144444
Q 841*30  T 25230  ☒ 148000
Q 463*53  T 24539  ☒ 135500
Q 631*664 T 418984 ☒ 144444
Q 78*356  T 27768  ☒ 135500
Q 9*465   T 4185   ☒ 140000

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 437us/step - loss: 1.7854 - acc: 0.1800 - val_loss: 1.7902 - val_acc: 0.1757
Q 739*47  T 34733  ☒ 111990
Q 827*680 T 562360 ☒ 102000
Q 136*74  T 10064  ☒ 111220
Q 39*657  T 25623  ☒ 105550
Q 739*56  T 41384  ☒ 105500
Q 410*859 T 352190 ☒ 105000
Q 952*25  T 23800  ☒ 105000
Q 124*31  T 3844   ☒ 11

18000/18000 [==============================] - 12s 691us/step - loss: 1.3148 - acc: 0.3977 - val_loss: 1.3077 - val_acc: 0.4117
Q 312*19  T 5928   ☒ 6552  
Q 685*3   T 2055   ☒ 2175  
Q 404*42  T 16968  ☒ 160560
Q 298*505 T 150490 ☒ 140750
Q 61*923  T 56303  ☒ 56009 
Q 665*513 T 341145 ☒ 370075
Q 617*75  T 46275  ☒ 460055
Q 846*984 T 832464 ☒ 776644
Q 55*579  T 31845  ☒ 397755
Q 42*425  T 17850  ☒ 160500

--------------------------------------------------
Iteration 14
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 11s 626us/step - loss: 1.2886 - acc: 0.4036 - val_loss: 1.2822 - val_acc: 0.4233
Q 548*54  T 29592  ☒ 297720
Q 703*20  T 14060  ☒ 136600
Q 205*17  T 3485   ☒ 3225  
Q 447*28  T 12516  ☒ 12776 
Q 517*115 T 59455  ☒ 577255
Q 78*63   T 4914   ☒ 4676  
Q 425*35  T 14875  ☒ 157750
Q 336*26  T 8736   ☒ 90022 
Q 41*573  T 23493  ☒ 22699 
Q 0*259   T 0      ☑ 0     

--------------------------------------------------
Iteratio

Q 402*51  T 20502  ☒ 20008 
Q 479*594 T 284526 ☒ 280116
Q 87*991  T 86217  ☒ 880077
Q 946*679 T 642334 ☒ 631436
Q 527*67  T 35309  ☒ 34411 
Q 36*953  T 34308  ☒ 345144
Q 293*82  T 24026  ☒ 240122
Q 664*237 T 157368 ☒ 150116
Q 879*263 T 231177 ☒ 234111
Q 95*550  T 52250  ☒ 522500

--------------------------------------------------
Iteration 27
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 18s 1ms/step - loss: 1.0963 - acc: 0.4797 - val_loss: 1.1385 - val_acc: 0.4485
Q 14*574  T 8036   ☒ 7124  
Q 598*94  T 56212  ☒ 589740
Q 923*105 T 96915  ☒ 906655
Q 552*26  T 14352  ☒ 137720
Q 540*840 T 453600 ☒ 447400
Q 239*161 T 38479  ☒ 367877
Q 608*883 T 536864 ☒ 515774
Q 96*295  T 28320  ☒ 276500
Q 220*76  T 16720  ☒ 167800
Q 78*624  T 48672  ☒ 47772 

--------------------------------------------------
Iteration 28
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 15s 854us/step - lo

18000/18000 [==============================] - 14s 803us/step - loss: 0.8755 - acc: 0.5829 - val_loss: 0.8979 - val_acc: 0.5861
Q 374*4   T 1496   ☒ 14760 
Q 3*289   T 867    ☒ 827   
Q 532*219 T 116508 ☒ 119908
Q 487*44  T 21428  ☒ 218880
Q 8*261   T 2088   ☑ 2088  
Q 510*20  T 10200  ☒ 102000
Q 698*2   T 1396   ☑ 1396  
Q 293*271 T 79403  ☒ 707831
Q 318*972 T 309096 ☒ 306236
Q 658*86  T 56588  ☒ 568486

--------------------------------------------------
Iteration 41
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 16s 889us/step - loss: 0.8666 - acc: 0.5876 - val_loss: 0.8984 - val_acc: 0.5732
Q 420*649 T 272580 ☒ 262080
Q 178*3   T 534    ☒ 594   
Q 94*805  T 75670  ☒ 756800
Q 954*54  T 51516  ☒ 508760
Q 482*867 T 417894 ☒ 417234
Q 21*16   T 336    ☑ 336   
Q 58*491  T 28478  ☒ 282784
Q 904*67  T 60568  ☒ 608880
Q 258*0   T 0      ☑ 0     
Q 984*193 T 189912 ☒ 183212

--------------------------------------------------
Iteratio

Q 672*71  T 47712  ☑ 47712 
Q 23*530  T 12190  ☒ 122900
Q 32*13   T 416    ☒ 456   
Q 52*74   T 3848   ☒ 3948  
Q 19*977  T 18563  ☒ 18503 
Q 601*4   T 2404   ☑ 2404  
Q 243*49  T 11907  ☒ 123777
Q 5*48    T 240    ☑ 240   
Q 427*887 T 378749 ☒ 389719
Q 792*41  T 32472  ☒ 32972 

--------------------------------------------------
Iteration 54
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 14s 781us/step - loss: 0.7798 - acc: 0.6121 - val_loss: 0.8288 - val_acc: 0.5889
Q 540*63  T 34020  ☒ 346200
Q 751*622 T 467122 ☒ 467362
Q 60*51   T 3060   ☒ 316004
Q 146*876 T 127896 ☒ 123996
Q 925*91  T 84175  ☒ 826257
Q 754*406 T 306124 ☒ 305484
Q 159*104 T 16536  ☒ 152360
Q 707*847 T 598829 ☒ 509719
Q 24*499  T 11976  ☒ 121168
Q 92*758  T 69736  ☒ 70636 

--------------------------------------------------
Iteration 55
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 12s 692us/step - 

...

18000/18000 [==============================] - 14s 790us/step - loss: 0.6596 - acc: 0.6592 - val_loss: 0.8312 - val_acc: 0.5990
Q 547*303 T 165741 ☒ 160121
Q 73*452  T 32996  ☒ 320762
Q 848*24  T 20352  ☒ 197120
Q 20*993  T 19860  ☒ 196600
Q 619*69  T 42711  ☒ 42511 
Q 633*2   T 1266   ☒ 1246  
Q 774*558 T 431892 ☒ 431532
Q 605*925 T 559625 ☒ 559175
Q 83*84   T 6972   ☒ 6032  
Q 36*950  T 34200  ☒ 348000

--------------------------------------------------
Iteration 95
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 15s 831us/step - loss: 0.6290 - acc: 0.6664 - val_loss: 0.8075 - val_acc: 0.6034
Q 189*62  T 11718  ☒ 11978 
Q 582*0   T 0      ☑ 0     
Q 27*729  T 19683  ☒ 196339
Q 859*267 T 229353 ☒ 226613
Q 392*76  T 29792  ☒ 293324
Q 825*71  T 58575  ☒ 585757
Q 61*968  T 59048  ☒ 591288
Q 700*996 T 697200 ☒ 703600
Q 664*950 T 630800 ☒ 624400
Q 706*356 T 251336 ☒ 254216

--------------------------------------------------
Iteratio

Q 615*266 T 163590 ☒ 160790
Q 259*5   T 1295   ☒ 1235  
Q 52*475  T 24700  ☒ 243000
Q 400*0   T 0      ☑ 0     
Q 906*289 T 261834 ☒ 268434
Q 168*63  T 10584  ☒ 10724 
Q 392*89  T 34888  ☒ 35048 
Q 26*691  T 17966  ☒ 189862
Q 332*650 T 215800 ☒ 210600
Q 135*29  T 3915   ☒ 3855  

--------------------------------------------------
Iteration 108
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 7s 413us/step - loss: 0.5906 - acc: 0.6822 - val_loss: 0.8471 - val_acc: 0.5975
Q 638*721 T 459998 ☒ 464718
Q 706*923 T 651638 ☒ 654778
Q 9*772   T 6948   ☒ 6968  
Q 18*625  T 11250  ☒ 114500
Q 615*266 T 163590 ☒ 162590
Q 76*903  T 68628  ☒ 691888
Q 700*293 T 205100 ☒ 200100
Q 36*642  T 23112  ☒ 235922
Q 7*686   T 4802   ☒ 4842  
Q 683*777 T 530691 ☒ 533471

--------------------------------------------------
Iteration 109
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 12s 674us/step -

18000/18000 [==============================] - 14s 754us/step - loss: 0.5489 - acc: 0.7036 - val_loss: 0.8730 - val_acc: 0.6026
Q 463*53  T 24539  ☒ 246892
Q 47*241  T 11327  ☒ 11007 
Q 647*488 T 315736 ☒ 311676
Q 4*945   T 3780   ☒ 37800 
Q 776*4   T 3104   ☒ 31440 
Q 555*723 T 401265 ☒ 402585
Q 24*631  T 15144  ☒ 149244
Q 28*361  T 10108  ☒ 1008  
Q 672*16  T 10752  ☒ 106222
Q 9*620   T 5580   ☒ 558006

--------------------------------------------------
Iteration 122
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 13s 719us/step - loss: 0.5724 - acc: 0.6969 - val_loss: 0.8815 - val_acc: 0.6063
Q 372*49  T 18228  ☒ 18268 
Q 62*272  T 16864  ☒ 165444
Q 917*10  T 9170   ☒ 933000
Q 28*422  T 11816  ☒ 117160
Q 41*362  T 14842  ☒ 14162 
Q 595*225 T 133875 ☒ 138775
Q 26*333  T 8658   ☒ 8958  
Q 389*19  T 7391   ☒ 70017 
Q 972*12  T 11664  ☒ 119044
Q 478*80  T 38240  ☒ 370400

--------------------------------------------------
Iterati

Q 671*112 T 75152  ☒ 777722
Q 618*724 T 447432 ☒ 440512
Q 14*912  T 12768  ☒ 127680
Q 145*645 T 93525  ☒ 938750
Q 251*547 T 137297 ☒ 139877
Q 226*647 T 146222 ☒ 144442
Q 886*979 T 867394 ☒ 869374
Q 531*34  T 18054  ☒ 18334 
Q 410*859 T 352190 ☒ 354190
Q 352*841 T 296032 ☒ 298452

--------------------------------------------------
Iteration 135
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 17s 933us/step - loss: 0.5252 - acc: 0.7103 - val_loss: 0.9122 - val_acc: 0.6048
Q 21*963  T 20223  ☒ 195233
Q 11*63   T 693    ☒ 653   
Q 551*1   T 551    ☑ 551   
Q 43*521  T 22403  ☒ 222533
Q 697*0   T 0      ☑ 0     
Q 587*667 T 391529 ☒ 398349
Q 763*71  T 54173  ☒ 53283 
Q 355*961 T 341155 ☒ 337045
Q 565*52  T 29380  ☒ 29740 
Q 939*3   T 2817   ☒ 2747  

--------------------------------------------------
Iteration 136
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 11s 625us/step 

18000/18000 [==============================] - 12s 684us/step - loss: 0.4730 - acc: 0.7331 - val_loss: 0.9328 - val_acc: 0.6014
Q 78*966  T 75348  ☒ 754086
Q 300*392 T 117600 ☑ 117600
Q 829*662 T 548798 ☒ 549718
Q 724*94  T 68056  ☒ 676360
Q 26*333  T 8658   ☒ 8758  
Q 42*830  T 34860  ☒ 342600
Q 23*530  T 12190  ☒ 129900
Q 9*620   T 5580   ☒ 558006
Q 46*90   T 4140   ☒ 41400 
Q 700*156 T 109200 ☒ 101200

--------------------------------------------------
Iteration 149
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 19s 1ms/step - loss: 0.4602 - acc: 0.7405 - val_loss: 0.9308 - val_acc: 0.6043
Q 68*567  T 38556  ☒ 385560
Q 392*76  T 29792  ☒ 29392 
Q 81*870  T 70470  ☒ 70670 
Q 52*805  T 41860  ☒ 414600
Q 75*310  T 23250  ☒ 227500
Q 902*797 T 718894 ☒ 724674
Q 23*716  T 16468  ☒ 160088
Q 72*871  T 62712  ☒ 627326
Q 620*731 T 453220 ☒ 456420
Q 538*8   T 4304   ☒ 43046 


### Testing

In [43]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
right = 0
predictions = model.predict_classes(test_x, verbose = 1) 
for i in range(60000):
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(predictions[i], calc_argmax=False)
    if correct == guess:
        right = right + 1

acc = right/60000
print("Accuracy : ", acc)
#####################################################

MSG : Prediction
60000/60000 [==============================] - 23s 386us/step
Accuracy :  0.049166666666666664



|Layers|TRAINING_SIZE|Iterarions|Validation Accuracy|Test Accuracy|
|---|---|---|---|---|
|1|80000|150|0.5760|0.0425|
|2|80000|150|0.6043|0.0492|

由上表中的Validation Accuracy及Test Accuracy可以看出模型在乘法的效果很不好，而從Test Accuracy小於Validation Accuracy很多也能推測模型可能有over fitting的問題。要改善模型可以從增加樣本數或更改模型的架構去著手。